In [45]:
import battery_parser as bp
import os
import pandas as pd
import numpy as np
import re
import copy

In [12]:
borders = {'LFP':[2.51,3.64, 280.0], 'NMC':[2.51, 4.19, 195.0],'LTO':[1.51, 2.84, 102.0]}


In [3]:
# from importlib import reload
# reload(bp)

In [10]:
class Check_pattern:  # TODO перенести в файл с обработкой (который будет)
    """
    Creates object that checks if given value list/dataframe corresponds to given pattern.
    Pattern engine use slicing window with size equal to pattern length. Each pattern element is dict in form {'column_name':(compare operation, value)} for one condition per column or {'column_name':[(compare operation1, value1), (compare operation2, value2)]} for multiple conditions for the column.
    Args:
        pattern:
    """

    def __init__(self, pattern:list, ):
        self.length = len(pattern)
        self.pattern = [self.transform_pattern(step) for step in pattern]


    def __call__(self, split: pd.DataFrame):
        """
        Checks if values in list are consistent with pattern. List length should be
        equal to pattern length.
        Args:
            split (): list of values.

        Returns:

        """
        assert len(self.pattern) == len(split)
        checks = [self.check_series(series[1], conditions)
                  for series, conditions
                  in zip(split.reset_index(drop=True).iterrows(), self.pattern)]
        return all(checks)

    def __len__(self):
        return len(self.pattern)

    @staticmethod
    def transform_pattern(step:dict):
        step_transform = []
        for key in step.keys():
            if len(step[key]) > 1 and isinstance(step[key], list):
                step_transform.extend([(key, *comparator ) for comparator in step[key]])
            else:
                step_transform.append((key, *step[key]))
        return step_transform


    def check_series(self, series, conditions):
        results = []
        for column, operator, value in conditions:
            result = self.check_condition(a=series[column], b=value, operation=operator)
            results.append(result)
        return all(results)

    @staticmethod
    def check_condition(a, b, operation):
        match operation:
            case 'less':
                return a < b
            case 'more':
                return a > b
            case 'equal':
                return a == b
            case 'contain':
                return b in a
            case 'approx':
                borders = [b*0.99, b*1.01]
                borders.sort()
                return (a >= borders[0]) and (a <= borders[1])


def window_slider(window_size, stat_data):
    len_data = len(stat_data)
    slices = []
    for i in range(len_data-window_size+1):
        slices.append(stat_data.iloc[i:i+window_size])
    return slices

def find_pattern(statistics: pd.DataFrame, pattern: list):
    pattern_checker = Check_pattern(pattern)
    window_size = len(pattern_checker)

    slices = window_slider(window_size, statistics)
    roll = {sl.iloc[0].name:pattern_checker(sl) for sl in slices}
    roll = pd.DataFrame.from_dict(roll, 'index', columns=['result'])
    windows = []
    for i in roll[roll['result']].index:
        start_index = i
        end_index =  start_index + window_size
        windows.append(list(range(start_index, end_index)))
    return windows

In [19]:
def create_pattern(e_min, e_max):
    p = {'discharge':[{'Step Type':('contain', 'dchg')}],
         'charge':[{'Step Type':('contain', 'chg')}],
        'discharge_total':[{'E_last':('less', e_min), 'Step Type':('contain', 'dchg')}],
         'charge_total':[{'E_last':('more', e_max), 'Step Type':('contain', 'chg')}],
         'rest':[{'Step Type':('contain','rest')}],
         'power':[{'Step Type':('contain','rest')}, {'Step Type':('contain','dchg'), 'Time_range':('equal', 10)},{'Step Type':('contain','rest')}],
         'R':[{'Step Type':('contain','dchg'), 'Time_range':('equal', 30)}, {'Step Type':('contain','dchg'), 'Time_range':('equal', 5)}]}

    return p

In [20]:
def create_dst(p_max):
    power_percent = [0.0, 0.125, 0.25, -0.125, 0.0,
                    0.125, 0.25, -0.125, 0.0, 0.125,
                    0.25, -0.125, 0.0, 0.125, 1.0,
                    0.625, -0.25, 0.25, -0.5, 0.0]
    power = [i*p_max for i in power_percent]
    time_a = [16,28,12,8,16,
            24,12,8,16,24,
            12,8,16,36,8,
            24,8,32,8,44]
    time_b = [16,28,12,8,16,
            24,12,8,16,24,
            12,8,16,36,8,
            120,8,32,8,44]

    dst_all = []
    for i in power:
        if i>0:
            step='cp dchg'
        elif i < 0:
            step = 'cp chg'
        elif i == 0:
            step = 'rest'
        one_move = {'Step Type':('contain', step), 'P_mean':('approx', -i)}
        dst_all.append(one_move)

    dst_b = []
    for step, time in zip(dst_all, time_b):
        times = {'Time_range':('equal', time)}
        times.update(step)
        dst_b.append(times)

    dst_a = []
    for step, time in zip(dst_all, time_a):
        times = {'Time_range':('equal', time)}
        times.update(step)
        dst_a.append(times)
    return dst_all, dst_a, dst_b


In [29]:
def step_1(data):
    test_result = {'Q_dch':data[data['Step Type'].str.contains(' dchg')].iloc[-1]['Q_max'],
                   'En_dch':data[data['Step Type'].str.contains(' dchg')].iloc[-1]['Energy_max'],
                   'Q_ch':data[data['Step Type'].str.contains(' chg')].iloc[-1]['Q_max'],
                   'En_ch':data[data['Step Type'].str.contains(' chg')].iloc[-1]['Energy_max'],
                   'Datetime':data.iloc[0]['Datetime_min']}
    test_result['Q_eff'] = test_result['Q_dch']/test_result['Q_ch']
    test_result['T_max'] = data['T_max'].max()
    test_result['index'] = data.index[0]
    test_result['En_eff'] = test_result['En_dch']/test_result['En_ch']
    return data.index[-1]+1, test_result slice1

def step_2(data):
    result = {}
    result['P_max'] = data[data['Step Type'].str.contains(' dchg')].iloc[-1]['P_last']
    return data.index[-1]+1, result

def step_2_1(data):
    _E = data['E_last']
    _I = data['I_mean']
    result = {'R':(_E.iloc[-1]-_E.iloc[-2])/(_I.iloc[-1]-_I.iloc[-2])}
    return data.index[-1]+1, result

def step_3(data):
    if data['Step Type'].nunique() != 3:
        print('dst step 3 wrong cut!')
    else:
        print('dst step 3 ok')
    result = {}
    result['full_dst'] = len(find_pattern(data, p['dst_all']))
    result['Q_dchg'] = -data[data['Net Q']<0]['Net Q'].sum()
    result['Q_chg'] = data[data['Net Q']>0]['Net Q'].sum()
    result['Q'] = result['Q_dchg'] - result['Q_chg']
    return data.index[-1]+1, result

# def step_3_1(data):
#     pass

In [30]:
def extract_results1(statistics_data):
    pattern = []
    for i  in p:
        if 'full_cycle' in i:
            pattern.extend(find_pattern(statistics_data, p[i]))
    results = {}
    data = statistics_data.loc[pattern[0]]
    start_idx, result = step_1(data)
    results['1'] = result

    pattern = find_pattern(statistics_data.loc[start_idx:], p['power'])
    data = statistics_data.loc[pattern[0]]
    start_idx, result = step_2(data)
    results['2'] = result

    pattern = find_pattern(statistics_data.loc[start_idx:], p['R'])
    data = statistics_data.loc[pattern[0]]
    start_idx, result = step_2_1(data)
    results['2.1'] = result

    start_search = find_pattern(statistics_data.loc[start_idx:], p['charge'])[0][0]
    end_search = find_pattern(statistics_data.loc[start_search:], p['discharge'])[0][0]
    data = statistics_data.loc[start_search+1:end_search]
    start_idx, result = step_3(data)
    results['3'] = result

    # pattern = p['discharge']
    return results

Разработка метода извлечения данных из файлов ndax - для пунктов 1- 4,1.
1. Задать папку, где лежат нужные эксперименты (в формате .xlsx)

1. полный цикл, извлечение данных

In [31]:
test1 = []
for i in os.walk(r'D:\!Science\Физтех\Циклирования\28.05.24 - результаты'):
    spl = i[0].split('\\')
    if '-4.1' in spl[-2] or '- п4.1' in spl[-2]:
        test1.append(i[0])


results_all = []
for i, directory in enumerate(test1):
  # Папка где лежат все эксперименты
    path_list = directory.split('\\')
    if 'NMC' in path_list:
        key = 'NMC'
    elif 'LTO' in path_list:
        key = 'LTO'
    elif 'LFP' in path_list:
        key = 'LFP'
    e_min, e_max, p_max = borders[key]
    dst_all, dst_a, dst_b = create_dst(p_max)
    p = create_pattern(e_min, e_max)
    p['dst_all'] = dst_all
    p['dst_b'] = dst_b
    p['dst_a'] = dst_a
    files = bp.list_files(directory=directory, filetype='xlsx')  # Список файлов с форматом xls из указанной папки
    data = [bp.import_xls(filename, data_name_pattern='record', temp_name_pattern='auxTemp', temp_column_pattern='T1') for filename in files]  # Список импортированных из этих файлов
    test_dataset = pd.concat(data,ignore_index=True)
    test_dataset.drop('DataPoint', axis=1, inplace=True) # у нас всего один файл - на нём и работаем
    bp.rename_columns(test_dataset)  # Изменяет названия некоторых колонок см. комментарии к функции
    test_dataset.rename(columns={'Power(W)':'P'}, inplace=True)
    bp.parse_time(test_dataset, time_column='Time', datetime_column='Datetime')  # Изменяет вид колонки времени на
    # секунды, и даты в формат datetime
    statistic_pattern = {'Step Type':'unique_values',
                         'I':['mean', 'std'],
                         'Time':['range', 'diff'],
                         'Q':['max'],
                         'E':['first', 'last','mean'],
                         'T':['mean', 'min', 'max'],
                         'Datetime':['min', 'max'],
                         'Energy':'max',
                         'P':['mean', 'last']}  # Можно задать формат статистики, возможные методы статистики см.в комментариях к bp.generate_statistics

    statistics_data = bp.generate_statistics(test_dataset,
                                             group_marker=(test_dataset['Time'] < test_dataset['Time'].shift()).cumsum(),
                                             statistics_pattern=statistic_pattern)  # Подбивает статистику по таблице
    statistics_data['Net Q'] = np.sign(statistics_data['I_mean'])*statistics_data['Q_max']
    statistics_data['Net Energy'] = np.sign(statistics_data['I_mean'])*statistics_data['Energy_max']
    statistics_data.rename(columns={'Step Type_unique_values':'Step Type'}, inplace=True)
    statistics_data['Step Type'] = statistics_data['Step Type'].str.lower()
    result = extract_results1(statistics_data)
    result['type'] = key
    result['filepath'] = files
    result['Number'] = i
    print(result)
    result['statistics'] = statistics_data
    results_all.append(result)

D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


dst step 3 ok
{'1': {'Q_dch': 48.6748, 'En_dch': 154.5293, 'Q_ch': 48.3056, 'En_ch': 164.3237, 'Datetime': Timestamp('2024-04-08 14:38:06'), 'Q_eff': 1.0076430061938988, 'T_max': 33.3, 'index': 0, 'En_eff': 0.9403956945954844}, '2': {'P_max': -283.8343}, '2.1': {'R': 0.001466933487564924}, '3': {'full_dst': 44, 'Q_dchg': 58.5287, 'Q_chg': 9.4807, 'Q': 49.048}, 'type': 'LFP', 'filepath': ['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №73 Старение+45 2-3\\1. Пункт 1-4.1\\№2\\BTS82-130-1-1-231-20240408143806-Программа п1-п4.1 C-LFP.xlsx'], 'Number': 0}


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


dst step 3 ok
{'1': {'Q_dch': 48.1755, 'En_dch': 153.067, 'Q_ch': 47.9617, 'En_ch': 162.9328, 'Datetime': Timestamp('2024-04-08 14:38:18'), 'Q_eff': 1.0044577235585894, 'T_max': 33.5, 'index': 0, 'En_eff': 0.9394486561330807}, '2': {'P_max': -284.004}, '2.1': {'R': 0.0014444503593525645}, '3': {'full_dst': 43, 'Q_dchg': 58.2802, 'Q_chg': 9.479099999999999, 'Q': 48.801100000000005}, 'type': 'LFP', 'filepath': ['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №73 Старение+45 2-3\\1. Пункт 1-4.1\\№3\\BTS82-130-1-2-231-20240408143819-Программа п1-п4.1 C-LFP.xlsx'], 'Number': 1}


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


...

dst step 3 ok
{'1': {'Q_dch': 17.6382, 'En_dch': 64.8659, 'Q_ch': 17.7534, 'En_ch': 67.63, 'Datetime': Timestamp('2024-03-25 10:26:02'), 'Q_eff': 0.993511102098753, 'T_max': 30.0, 'index': 0, 'En_eff': 0.9591290847257135}, '2': {'P_max': -192.807}, '2.1': {'R': 0.0018333915611215932}, '3': {'full_dst': 26, 'Q_dchg': 21.2101, 'Q_chg': 3.4249, 'Q': 17.7852}, 'type': 'NMC', 'filepath': ['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\NMC\\Заявка №64 C-NMC №014 №015\\1. Пункт 1-4.1\\015\\BTS82-131-1-6-2818575908-20240325102602-пункт 1-4.1.xlsx'], 'Number': 21}


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


dst step 3 ok
{'1': {'Q_dch': 17.7959, 'En_dch': 65.5125, 'Q_ch': 17.7652, 'En_ch': 67.6347, 'Datetime': Timestamp('2024-03-25 10:17:02'), 'Q_eff': 1.0017280976290726, 'T_max': 31.9, 'index': 0, 'En_eff': 0.9686226153143284}, '2': {'P_max': -192.8628}, '2.1': {'R': 0.0018475631123532133}, '3': {'full_dst': 26, 'Q_dchg': 21.2247, 'Q_chg': 3.4307, 'Q': 17.793999999999997}, 'type': 'NMC', 'filepath': ['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\NMC\\Заявка№ 61\\1. п 1- п4.1\\012\\BTS82-131-1-3-2818575908-20240325101703-пункт 1-4.1.xlsx'], 'Number': 22}


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


dst step 3 ok
{'1': {'Q_dch': 17.7674, 'En_dch': 65.4086, 'Q_ch': 17.79, 'En_ch': 67.7375, 'Datetime': Timestamp('2024-03-25 10:17:17'), 'Q_eff': 0.9987296233839236, 'T_max': 31.1, 'index': 0, 'En_eff': 0.9656187488466508}, '2': {'P_max': -192.7644}, '2.1': {'R': 0.0018335744396328893}, '3': {'full_dst': 26, 'Q_dchg': 21.2315, 'Q_chg': 3.429, 'Q': 17.802500000000002}, 'type': 'NMC', 'filepath': ['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\NMC\\Заявка№ 61\\1. п 1- п4.1\\013\\BTS82-131-1-4-2818575908-20240325101717-пункт 1-4.1.xlsx'], 'Number': 23}


In [41]:
data = results_all[20]['statistics']
pattern = []
for i  in p:
    if 'full_cycle' in i:
        pattern.extend(find_pattern(statistics_data, p[i]))
results = {}
data = statistics_data.loc[pattern[0]]
start_idx, result = step_1(data)
results['1'] = result

pattern = find_pattern(statistics_data.loc[start_idx:], p['power'])
data = statistics_data.loc[pattern[0]]
start_idx, result = step_2(data)
results['2'] = result

pattern = find_pattern(statistics_data.loc[start_idx:], p['R'])
data = statistics_data.loc[pattern[0]]
start_idx, result = step_2_1(data)
results['2.1'] = result

start_search = find_pattern(statistics_data.loc[start_idx:], p['charge'])[0][0]
end_search = find_pattern(statistics_data.loc[start_search:], p['discharge'])[0][0]
data = statistics_data.loc[start_search+1:end_search]
start_idx, result = step_3(data)
results['3'] = result
results

dst step 3 ok


{'1': {'Q_dch': 17.7674,
  'En_dch': 65.4086,
  'Q_ch': 17.79,
  'En_ch': 67.7375,
  'Datetime': Timestamp('2024-03-25 10:17:17'),
  'Q_eff': 0.9987296233839236,
  'T_max': 31.1,
  'index': 0,
  'En_eff': 0.9656187488466508},
 '2': {'P_max': -192.7644},
 '2.1': {'R': 0.0018335744396328893},
 '3': {'full_dst': 26,
  'Q_dchg': 21.2315,
  'Q_chg': 3.429,
  'Q': 17.802500000000002}}

In [57]:

for i, directory in enumerate(test1):
  # Папка где лежат все эксперименты
    path_list = directory.split('\\')
    if 'NMC' in path_list:
        key = 'NMC'
    elif 'LTO' in path_list:
        key = 'LTO'
    elif 'LFP' in path_list:
        key = 'LFP'
    e_min, e_max, p_max = borders[key]
    dst_all, dst_a, dst_b = create_dst(p_max)
    p = create_pattern(e_min, e_max)
    p['dst_all'] = dst_all
    p['dst_b'] = dst_b
    p['dst_a'] = dst_a
    files = bp.list_files(directory=directory, filetype='xlsx')  # Список файлов с форматом xls из указанной папки
    print(files)

['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №73 Старение+45 2-3\\1. Пункт 1-4.1\\№2\\BTS82-130-1-1-231-20240408143806-Программа п1-п4.1 C-LFP.xlsx']
['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №73 Старение+45 2-3\\1. Пункт 1-4.1\\№3\\BTS82-130-1-2-231-20240408143819-Программа п1-п4.1 C-LFP.xlsx']
['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №74 Старение +45 4-5\\1. Пункт 1-4.1\\4\\BTS82-130-1-3-231-20240408144813-Программа п1-п4.1 C-LFP.xlsx']
['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №74 Старение +45 4-5\\1. Пункт 1-4.1\\5\\BTS82-130-1-4-231-20240408144824-Программа п1-п4.1 C-LFP.xlsx']
['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №75 Старение +60\\1. Пункт 1-4.1\\№6\\BTS82-130-1-5-231-20240408145729-Программа п1-п4.1 C-LFP.xlsx']
['D:\\!Science\\Физтех\\Циклирования\\28.05.24 - результаты\\LFP\\Заявка №75 Старение +60\\1. Пункт 1-4.1\\№7\\BT

In [63]:
len(test1)


24

In [61]:

channel = '-(13[0-9])-.-([0-9])'
a= re.findall(channel, files[0])
a

[('131', '4')]